In [1]:
#to open cred files
import os
import yaml
#cleaning and operations
import pandas as pd
import numpy as np
#to read translations
import json
import unidecode

In [2]:
#bigquery
import pandas_gbq
from google.cloud import bigquery

In [4]:
#FIREBASE
import firebase_admin
#Firbease (pip install firebase_admin)
from firebase_admin import credentials
from firebase_admin import firestore

In [3]:
from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/contoan/.env')

True

In [ ]:
## OPEN CONECTION TO BIGQUERRY
client = bigquery.Client()
dataset_id = "{}.contoan".format(client.project)
dataset = bigquery.Dataset(dataset_id)

In [5]:
filename=os.environ['FIREBASE_FILENAME']

In [6]:
#OPEN CONECTION TO FIREBASE
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

## Accounting table

### GET DATA

In [7]:
#get all the collection
col_query = db.collection('Accounting').stream()

In [8]:
#save it in a list of dictionaries
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [9]:
#convert to dataframe
accounting = pd.DataFrame(accounts)

In [10]:
accounting['amount']=pd.to_numeric(accounting['amount'])

In [11]:
#nulls. size and datatype
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1929 entries, 0 to 1928
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_in_charge  1929 non-null   object 
 1   creation_date   1929 non-null   object 
 2   creator_user    1929 non-null   object 
 3   phase           617 non-null    object 
 4   execution_date  1929 non-null   object 
 5   project         1929 non-null   object 
 6   type            1929 non-null   object 
 7   vat_amount      1860 non-null   float64
 8   images          1926 non-null   object 
 9   place           1929 non-null   object 
 10  description     1885 non-null   object 
 11  account_id      1929 non-null   object 
 12  concept         1929 non-null   object 
 13  id              1929 non-null   object 
 14  origin          1929 non-null   object 
 15  target_id       1929 non-null   object 
 16  vat             1706 non-null   float64
 17  amount          1929 non-null   f

In [12]:
#nulls heatmap - import seaborn as sns
#sns.heatmap(accounting.isna())

### CLEAN DATA

In [13]:
#lets see how it shows
accounting.head()

,user_in_charge,creation_date,creator_user,phase,execution_date,project,type,vat_amount,images,place,description,account_id,concept,id,origin,target_id,vat,amount,intervention,code
0,IjBxXuBshlfq2MUzwBSK,2020-08-26T18:02:55.591Z,z5m936GA0t3vHM28QKhR,None,2020-04-28T12:07:49.000Z,0DmODGTOEiM5lg9SGx0J,income,3.644628,[],spain,venta online: 2288996286563,3deqBmrZ1iJkAwMmxSOp,venta shopify,01WVDHF0gSSg1PCFsosk,script_dani,yfy4cPxkmFwIYVnoWD0A,21.0,21.00,w04441aFcU5b7pQm6Rd2,899
1,IjBxXuBshlfq2MUzwBSK,2020-12-07T08:56:28.137Z,z5m936GA0t3vHM28QKhR,None,2020-12-07T08:56:28.137Z,0DmODGTOEiM5lg9SGx0J,movement,0.000000,[],spain,2875805401187,FHwFsBItyEneaZh8yP0T,Gastos de envio Nacional,01nQGx25hZLez38FpWEe,shopify,iA9Pzv2CImjItzwCaQv0,0.0,6.10,w04441aFcU5b7pQm6Rd2,1959
2,IjBxXuBshlfq2MUzwBSK,2020-08-26T18:02:55.591Z,z5m936GA0t3vHM28QKhR,None,2020-04-28T19:50:24.000Z,0DmODGTOEiM5lg9SGx0J,income,6.595041,[],spain,venta online: 2290422218851,3deqBmrZ1iJkAwMmxSOp,venta shopify,026RXV3hyl33ussoU7D9,script_dani,iA9Pzv2CImjItzwCaQv0,21.0,38.00,w04441aFcU5b7pQm6Rd2,1049
3,IjBxXuBshlfq2MUzwBSK,2020-12-05T22:24:00.338Z,z5m936GA0t3vHM28QKhR,LXspD4R7rZ6Jw5EOEZYR,2020-12-05T22:24:00.338Z,0DmODGTOEiM5lg9SGx0J,expense,0.000000,[],spain,2872447467619,j2UYEKf01eFHMPZv9qHq,Comision Stripe,02EE48NlGSbqGrrLvIHl,shopify,iA9Pzv2CImjItzwCaQv0,0.0,1.20,w04441aFcU5b7pQm6Rd2,1934
4,uCF6bT532woZwE9vQOuv,2020-08-21T11:18:01.503Z,uCF6bT532woZwE9vQOuv,Dck0Vf7yfdGlBct3qoLY,2020-08-21T11:18:01.503Z,RaIH6dfSXt4V7vm6jmVE,expense,0.000000,[],spain,Parte pagada por OAN International de piezas d...,zMHMNTkAt43M0MmmRxJP,Piezas bomba Vergnet,02LozVMZvW51bygJ5e3O,contoan,vJbbj1kPxkcdXJyBOf1l,0.0,132.65,SEDR5KRqPQVF6qovZiVq,417


#### convert execution to isostring

In [14]:
def clean_execution(x):
    if(len(x['execution_date'])<15):
        doc_id = x['id']
        exec_date=x['execution_date']+'T14:14:58.593Z'
        print(exec_date)
        print(len(x['execution_date']))
        #upload to firebase if whished
        #a=db.collection(u'Accounting').document(doc_id).update({'execution_date':exec_date})
        
        return exec_date
    else:
        return x['execution_date']
        

In [15]:
accounting['execution_date']=accounting.apply(lambda x: clean_execution(x) ,axis=1)

2020-10-31T14:14:58.593Z
10
2020-11-25T14:14:58.593Z
10
2020-09-30T14:14:58.593Z
10
2020-09-01T14:14:58.593Z
10
2020-10-15T14:14:58.593Z
10
2020-10-31T14:14:58.593Z
10


##### clean VAT and replace nan with 0

In [16]:
def clean_vat_amount(x):
    doc_id = x['id']
    amount=x['amount']
    origin = x['origin']
    if origin =='script_dani':
        if x['vat']>0:
            vat=21
        else:
            vat=0
        #a=db.collection(u'Accounting').document(doc_id).update({'vat':vat})
    else:
        vat=x['vat']
        if not(vat >=0):
            vat=0
    vat_amount=amount*(1-1/(1+vat/100))

    #a=db.collection(u'Accounting').document(doc_id).update({'vat_amount':vat_amount})

    return pd.Series({'vat':vat,'vat_amount':vat_amount})

In [17]:
accounting[['vat','vat_amount']] = accounting.apply(lambda x: clean_vat_amount(x),axis=1)

###### convert dates to datetime

In [18]:
#utc=True important if dates are in diferent formats
accounting['creation_date'] = pd.to_datetime(accounting['creation_date'],utc=True)
accounting['execution_date'] = pd.to_datetime(accounting['execution_date'],utc=True)

In [19]:
def getimages(x):
    if not (x):
        return ''
    elif(len(x)==0):
        return ''
    elif('download_url' in x[0].keys()):
        download_url = x[0]['download_url']
        if(download_url == None):
            return ''
        else:
            return download_url
    else:
        return ''

In [20]:
accounting['receipt']=accounting.images.apply(lambda x: getimages(x))

##### double check

In [21]:
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1929 entries, 0 to 1928
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   user_in_charge  1929 non-null   object             
 1   creation_date   1929 non-null   datetime64[ns, UTC]
 2   creator_user    1929 non-null   object             
 3   phase           617 non-null    object             
 4   execution_date  1929 non-null   datetime64[ns, UTC]
 5   project         1929 non-null   object             
 6   type            1929 non-null   object             
 7   vat_amount      1929 non-null   float64            
 8   images          1926 non-null   object             
 9   place           1929 non-null   object             
 10  description     1885 non-null   object             
 11  account_id      1929 non-null   object             
 12  concept         1929 non-null   object             
 13  id              1929 non-null   o

### select columns

In [22]:
#and the order whe want to upload to bigquery
df = accounting[['concept', 'place', 'creation_date', 'vat', 'description', 'amount',
       'id', 'phase', 'creator_user', 'execution_date',
       'user_in_charge', 'origin', 'account_id', 'project', 'code', 'type',
       'intervention', 'target_id', 'vat_amount', 'receipt']]

## upload to BIGQUERY

In [23]:
client = bigquery.Client()

In [24]:
dataset_id = "{}.contoan".format(client.project)

In [25]:
dataset = bigquery.Dataset(dataset_id)

In [26]:
table_id = "{}.accounting".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

1it [00:06,  6.22s/it]


### SUBIR VENTAS

#### get data

In [353]:
col_query = db.collection('Nikarit_Sales').stream()

In [354]:
ventas = []
for acc in col_query:
    ventas.append(acc.to_dict())

In [355]:
df_ventas = pd.DataFrame(ventas)

In [356]:
df_ventas.shape

(846, 57)

### view

In [357]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       846 non-null    object 
 1   client_gender            846 non-null    object 
 2   product_movement         203 non-null    object 
 3   buyer_accepts_marketing  216 non-null    object 
 4   balsamo_unit_price       846 non-null    object 
 5   total_tax                846 non-null    float64
 6   crema_nb                 846 non-null    int64  
 7   manteca_unit_price       846 non-null    object 
 8   close_date               697 non-null    object 
 9   packContigo_nb           846 non-null    int64  
 10  subtotal_price           846 non-null    object 
 11  packHidra_nb             846 non-null    int64  
 12  client_last_name         836 non-null    object 
 13  country                  759 non-null    object 
 14  order_status_url         8

#### transform data

In [358]:
cols = list(df_ventas.columns)

In [359]:
cambios = [col for col in cols if 'price' in col]

In [360]:
for cambio in cambios:
    df_ventas[cambio]=pd.to_numeric(df_ventas[cambio])

In [361]:
df_ventas['creation_date']=pd.to_datetime(df_ventas['creation_date'],utc=True)
df_ventas['close_date']=pd.to_datetime(df_ventas['close_date'],utc=True)

In [362]:
#double check
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 57 columns):
 #   Column                   Non-Null Count  Dtype              
---  ------                   --------------  -----              
 0   id                       846 non-null    object             
 1   client_gender            846 non-null    object             
 2   product_movement         203 non-null    object             
 3   buyer_accepts_marketing  216 non-null    object             
 4   balsamo_unit_price       846 non-null    float64            
 5   total_tax                846 non-null    float64            
 6   crema_nb                 846 non-null    int64              
 7   manteca_unit_price       846 non-null    float64            
 8   close_date               697 non-null    datetime64[ns, UTC]
 9   packContigo_nb           846 non-null    int64              
 10  subtotal_price           846 non-null    float64            
 11  packHidra_nb             846 non

In [363]:
df_ventas.shape

(846, 57)

### METER MUJERES Y HOMBRES

In [364]:
#client_gender

In [365]:
#lista de mujeres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/mujeres.csv'
df_mujeres = pd.read_csv(url, error_bad_lines=False)
l_mujeres = df_mujeres['nombre'].to_list()

In [366]:
l_mujeres = [str(nom).lower() for nom in l_mujeres]

In [367]:
l_mujeres= l_mujeres+['maasuncion','mirenchu','begona','nadai','sra.','mcarmen','carmena','maita',
'madolores','reyero','ma','maconcepción','cris']

In [368]:
#lista de hombres
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/hombres.csv'
df_hombres = pd.read_csv(url, error_bad_lines=False)
l_hombres = df_hombres['nombre'].to_list()

In [369]:
l_hombres = [nom.lower() for nom in l_hombres]

In [370]:
l_hombres= l_hombres+['fcojavier']

In [371]:
#aplicar sexo
df_ventas['client_gender'] = df_ventas['client_name'].apply(lambda x: 
                            'F' if unidecode.unidecode(
                                    str(x).replace(" ", "").lower()) in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x).replace(" ", "").lower())  in l_hombres
                            else 'F' if unidecode.unidecode(
                                str(x).split(" ")[0].lower())  in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x).split(" ")[0].lower())  in l_hombres

                            else 'N/A')

In [372]:
#ver que sale
df_ventas['client_gender'].value_counts() # A MEJORRAR

F      659
M      164
N/A     23
Name: client_gender, dtype: int64

In [373]:
df_ventas[df_ventas['client_gender'] == 'N/A'
         ]['client_name'].apply(lambda x: unidecode.unidecode(str(x).replace(" ", "").lower()))

55                  chinwe
65              herguedas,
66        notariaserrano19
76       impulsandolab,s.l
104         beaaccemmurcia
310         farmacialaluna
315                herrera
320                 gandou
330                   none
388                  durga
406                   none
410                   none
417                   none
426       laposadademanolo
452                   none
471            ventasorteo
480              ventajose
504                   none
570                 clavis
581    herbolariosalamanca
752               vinuales
797                   none
840                   saez
Name: client_name, dtype: object

## METER DONACIONES

In [374]:
df_ventas['origin'].unique()

array(['Shopify', 'Contoan'], dtype=object)

In [375]:
def don(x):
    total_price = x['total_price']
    subtotal_price = x['subtotal_price']
    fecha = str(x['creation_date']).split(' ')[0]
    origin = x['origin']
    don = 0.0
    if '2020-11-01' < fecha and origin == "Shopify":
        if total_price < 40:
            don = total_price - subtotal_price - 5.50
        else:
            don = total_price - subtotal_price
            
    return don

df_ventas['donation'] = df_ventas.apply(lambda x: don(x),axis=1)

In [376]:
df_ventas['donation'] = df_ventas.apply(lambda x: don(x),axis=1)

In [377]:
df_ventas['donation'].sum()

220.73

## ERROR EN PROVINCE

In [378]:
import requests

In [379]:
shop_url = os.environ['SHOPIFY_ACCESS_URL']
dates = ['2019-01-01T00:15:47-04:00','2020-01-01T00:15:47-04:00',
         '2020-05-01T00:15:47-04:00','2020-12-10T00:15:47-04:00', '2021-01-10T00:15:47-04:00']
l_orders = []

for date_min,date_max in zip(dates[:-1],dates[1:]):
    orders_url = shop_url+'/orders.json'
    r = requests.get(orders_url,
                 params={
                        'limit':250,
                        'status':'any',
                         'created_at_max':date_max,
                         'created_at_min':date_min
                        })
    orders = pd.DataFrame(r.json()['orders'])
    l_orders.append(orders)
df_orders = pd.concat(l_orders,axis=0).reset_index().drop_duplicates(subset=['id'])

In [ ]:
df_orders['province_2'] = df_orders['billing_address'].apply(lambda x: x['province'])
df_orders['order_id'] = df_orders['id'].apply(str)
df_ventas['order_id'] = df_ventas['order_id'].apply(str)
df_ventas = pd.merge(df_ventas,df_orders[['order_id','province_2']],on='order_id',how='left')

df_ventas['province'] = df_ventas['province_2'] 
df_ventas = df_ventas.drop('province_2',axis=1)

In [388]:
cols = list(df_ventas.columns)
cols.pop(cols.index('to'))
cols.pop(cols.index('message'))
cols.pop(cols.index('navidadCode'))

'navidadCode'

In [389]:
df_ventas = df_ventas[cols]

In [390]:
df_ventas.shape

(846, 54)

In [391]:
df_ventas['province'].value_counts().head()

Madrid        427
Barcelona      45
Valladolid     42
Valencia       30
Málaga         22
Name: province, dtype: int64

## ACTUALIZAR FIREBAE

In [392]:
#l_ship_det = ['ship_'+a for a in list(df_ord['shipping_address'][0].keys())]
#df_ord['ship_zip']=df_ord['ship_zip'].fillna(value='')
#def update_firebase(x):
#    order_id = x['order_id']
#    if order_id==None :
#        return True
#    else:
#        order_id = int(order_id)
#    ship_zip=df_ord[df_ord['id']==order_id]['ship_zip'].values[0]
#    fire_id=x['id']
#    #print(x['creation_date'])
#    #a=db.collection(u'Nikarit_Sales').document(x['id']).update({'zip':ship_zip})
##df_ventas[df_ventas['origin']!='Contoan'].apply(lambda x: update_firebase(x),axis=1)

## CORREGIR SUBTOTAL Y IVA

In [393]:
def iva(x):
    iva = x['total_tax']
    
    #Corregir ventas de contoan
    if x['origin'] == 'Contoan':
        iva = x['total_price']*(1-1/1.21)
    return iva

df_ventas["total_tax_2"] = df_ventas.apply(lambda x: iva(x),axis=1)
df_ventas["total_tax"] = df_ventas["total_tax_2"]
df_ventas = df_ventas.drop('total_tax_2',axis=1)

In [398]:
def subtotal(x):
    subtotal = x['subtotal_price']
    
    #Corregir ventas de contoan
    if x['origin'] == 'Contoan':
        subtotal = x['total_price']
    return subtotal

df_ventas["subtotal_price_2"] = df_ventas.apply(lambda x: subtotal(x),axis=1)
df_ventas["subtotal_price"] = df_ventas["subtotal_price_2"]
df_ventas = df_ventas.drop('subtotal_price_2',axis=1)

In [402]:
#subir a bigquery
table_id = "contoan.sales"

pandas_gbq.to_gbq(df_ventas, table_id, project_id=client.project, if_exists='replace')

1it [00:07,  7.25s/it]


## TRANSFORM BOOLEAN

In [263]:
sql = """
CREATE OR REPLACE TABLE `oan-nikarit.contoan.sales` AS
SELECT
  * EXCEPT (buyer_accepts_marketing,taxes_included),
  CAST(buyer_accepts_marketing AS BOOL) AS buyer_accepts_marketing,
  CAST(taxes_included AS BOOL) AS taxes_included
FROM
  `oan-nikarit.contoan.sales`
"""

In [264]:
client.query(sql)

## Cuentas

In [7]:
#get all the collection
col_query = db.collection('Accounts').stream()

In [8]:
#save it in a list of dictionaries
cuentas = []
for acc in col_query:
    cuentas.append(acc.to_dict())

In [9]:
#convert to dataframe
df_accounts = pd.DataFrame(cuentas)

In [34]:
df_cuentas = df_accounts[['id','name','number','type','user_id']].copy()

In [35]:
with open('fr.json', 'r') as f:
    trads = json.load(f)

In [36]:
df_cuentas['name_fr'] = df_cuentas['name'].apply(lambda x: trads[x])

In [37]:
df_cuentas[df_cuentas['name_fr'].isnull()]

,id,name,number,type,user_id,name_fr


In [40]:
table_id = "{}.accounts".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_cuentas, table_id, project_id=client.project, if_exists='replace')

1it [00:05,  5.93s/it]


### PROYECTOS

In [41]:
#get all the collection
col_query = db.collection('Projects').stream()

In [42]:
#save it in a list of dictionaries
projects = []
for acc in col_query:
    projects.append(acc.to_dict())

In [43]:
#convert to dataframe
df_projects = pd.DataFrame(projects)

In [44]:
df_projectos=df_projects[['id','title','description','department','type','project_id','intervention_id']]

In [45]:
df_projectos.columns = ['id', 'name', 'description', 'department', 'type', 'project_id',
       'intervention_id']

In [46]:
table_id = "{}.projects".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_projectos, table_id, project_id=client.project, if_exists='replace')

1it [00:03,  3.59s/it]


### USUARIOS

In [50]:
#get all the collection
col_query = db.collection('Users').stream()

In [51]:
#save it in a list of dictionaries
usuarios = []
for acc in col_query:
    usuarios.append(acc.to_dict())

In [52]:
#convert to dataframe
df_usuarios = pd.DataFrame(usuarios)

In [53]:
df_users = df_usuarios[['id','name','last_name','email']]

In [54]:
df_users_final = pd.merge(df_users,df_cuentas[['number','user_id']],left_on='id',right_on='user_id',how='left')

In [55]:
df_users = df_users_final[['id','name','last_name','email','number']]

In [56]:
df_users.columns = ['id','name','last_name','email','cuenta']

In [58]:
table_id = "{}.users".format(dataset.dataset_id)

pandas_gbq.to_gbq(df_users, table_id, project_id=client.project, if_exists='replace')

1it [00:06,  6.33s/it]


##### september 2020 backup

In [67]:
table_id = "{}.accounting_202009".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

1it [00:04,  4.48s/it]


##### Jan 2021 backup

In [351]:
sql = """
CREATE TABLE AS `oan-nikarit.contoan.accounting_20210110`
AS
SELECT *
FROM `oan-nikarit.contoan.accounting`
"""

In [352]:
client.query(sql)